In [ ]:
# create schema raios.Ocorrencias

# select * from  raios.Ocorrencias.dados

# CREATE TABLE raios.Ocorrencias.dados (
#     Cod_Municipio VARCHAR,
#     Quantidade_de_Ocorrencias VARCHAR,
#     Nome_Municipio VARCHAR,
#     Janeiro VARCHAR,
#     Fevereiro VARCHAR,
#     Marco VARCHAR,
#     Abril VARCHAR,
#     Maio VARCHAR,
#     Junho VARCHAR,
#     Julho VARCHAR,
#     Agosto VARCHAR,
#     Setembro VARCHAR,
#     Outubro VARCHAR,
#     Novembro VARCHAR,
#     Dezembro VARCHAR,
#     UF VARCHAR,
#     Total VARCHAR,
#     Municipio_Estado VARCHAR
# )
# WITH (
#     csv_separator = ',',
#     external_location = 's3a://raios/Ocorrencias',
#     format = 'CSV',
#     skip_header_line_count = 1

# );

In [66]:
from trino.dbapi import connect
from trino.auth import BasicAuthentication

# Conectar ao Trino
conn = connect(
    host="https://trino.ifpbapps.online",
    port=443,
    user="trino",
    auth=BasicAuthentication("trino", "iGejndrYQkbWMHOcej7MWHoAJA7myZ6VFCgiXlfwC2bb8/8sRr3haRJk"),
    catalog="raios",
    schema="descargas",
)

# Criar um cursor
cur = conn.cursor()

# Executar a consulta
cur.execute("SELECT * FROM raios.descargas.dados")

rows = cur.fetchall()

for row in rows:
    print(row)

cur.close()
conn.close()

## Print das linhas só pra teste de conexão com o trino

['1718204', '1071', 'Porto Nacional', '15414', '16662', '1480', '688', '150', '0', '0', '422', '264', '6163', '0', '0.0', 'Tocantins', '41243.0', 'Porto Nacional - Tocantins']
['5103403', '984', 'Cuiabá', '7223', '5124', '3176', '1033', '767', '0', '0', '152', '1670', '5028', '9', '0.0', 'Mato Grosso', '24182.0', 'Cuiabá - Mato Grosso']
['5002704', '950', 'Campo Grande', '20469', '16216', '14488', '7552', '1373', '0', '43', '4688', '1884', '9728', '9', '0.0', 'Mato Grosso do Sul', '76450.0', 'Campo Grande - Mato Grosso do Sul']
['5103379', '920', 'Cotriguaçu', '3020', '11266', '3030', '1505', '3295', '37', '58', '2240', '5037', '14886', '0', '0.0', 'Mato Grosso', '44374.0', 'Cotriguaçu - Mato Grosso']
['5103254', '860', 'Colniza', '13920', '15129', '10922', '11244', '10340', '0', '134', '3490', '20218', '31406', '297', '0.0', 'Mato Grosso', '117100.0', 'Colniza - Mato Grosso']
['1721000', '789', 'Palmas', '11575', '7457', '762', '160', '154', '0', '0', '351', '282', '3602', '0', '0.0',

In [38]:
import psycopg2
from psycopg2 import sql
from trino.dbapi import connect
from trino.auth import BasicAuthentication

# Conectar ao banco de dados Trino
trino_conn = connect(
    host="https://trino.ifpbapps.online",
    port=443,
    user="trino",
    auth=BasicAuthentication("trino", "iGejndrYQkbWMHOcej7MWHoAJA7myZ6VFCgiXlfwC2bb8/8sRr3haRJk"),
    catalog="raios",
    schema="ocorrencias",
)
trino_cur = trino_conn.cursor()

# Obter informações sobre a estrutura da tabela no Trino
trino_cur.execute("DESCRIBE raios.ocorrencias.dados")
columns = trino_cur.fetchall()

# Conectar ao banco de dados Yugabyte
yugabyte_conn = psycopg2.connect(
    host="yugabyte.ifpbapps.online",
    port=5433,
    user="yugabyte",
    password="k2jed9i2hy3y0h9she87g3hgf879d8duoaysod72d2ad",
    database="raios"
)

# Criar a tabela se não existir
yugabyte_cur = yugabyte_conn.cursor()
create_table_statement = f"CREATE TABLE IF NOT EXISTS ocorrencias ("
for column_info in columns:
    column_name, data_type = column_info[:2]
    create_table_statement += f'"{column_name}" {data_type}, '
create_table_statement = create_table_statement.rstrip(', ') + ");"
yugabyte_cur.execute(create_table_statement)
yugabyte_conn.commit()

# Obter todos os dados da tabela no Trino
trino_cur.execute("SELECT * FROM raios.ocorrencias.dados")
data = trino_cur.fetchall()

# Inserir os dados na tabela no Yugabyte
columns_names = [column_info[0] for column_info in columns]
insert_statement = sql.SQL("INSERT INTO ocorrencias ({}) VALUES ({})").format(
    sql.SQL(', ').join(map(sql.Identifier, columns_names)),
    sql.SQL(', ').join(sql.Placeholder() * len(columns_names))
)

yugabyte_cur.executemany(insert_statement, data)
yugabyte_conn.commit()

# Fechar as conexões
trino_cur.close()
trino_conn.close()
yugabyte_cur.close()
yugabyte_conn.close()
